# SARSA 구현
환경이 바뀐 것이 아니라 에이전트의 업데이트 방식이 바뀐 것이기 때문에 06-1에서 사용한 GridWorld의 코드를 그대로 사용한다.

## 라이브러리 import 및 GridWorld 클래스

In [1]:
import random
import numpy as np # q(s,a)를 numpy array 형태로 관리하기 위함

In [2]:
class GridWorld():
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a == 0:
            self.move_left()
        elif a == 1:
            self.move_up()
        elif a == 2:
            self.move_right()
        elif a == 3:
            self.move_down()
        
        reward = -1 # 보상은 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        if self.y == 1 and self.x in [0, 1, 2]:
            pass
        elif self.y == 3 and self.x in [2, 3, 4]:
            pass
        elif self.y == 6:
            pass
        else:
            self.y += 1
        
    def move_left(self):
        if self.y == 0:
            pass
        elif self.y == 3 and self.x in [0, 1, 2]:
            pass
        elif self.y == 5 and self.x in [2, 3, 4]:
            pass
        else:
            self.y -= 1
    
    def move_up(self):
        if self.x == 0:
            pass
        elif self.x == 3 and self.y == 2:
            pass
        else:
            self.x -= 1
            
    def move_down(self):
        if self.x == 4:
            pass
        elif self.x == 1 and self.y == 4:
            pass
        else:
            self.x += 1
    
    def is_done(self):
        if self.x == 4 and self.y == 6: # 목표 지점인 (4, 6)에 도달
            return True
        else:
            return False
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

## QAgent 클래스
QAgent 클래스 역시 나머지 코드는 MC에서 짰던 코드와 같지만, `update_table` 함수가 조금 바뀌었다.
MC에서의 `update_table`은 에이전트가 경험한 history 전체를 인자로 받았지만, 이제는 **transition**을 인풋으로 받는다. 상태 $s$에서 $a$를 행해서 보상 $r$을 받고 상태 $s'$에 도달했다면, $(s,a,r,s')$가 하나의 트랜지션이다.

In [9]:
class QAgent():
    def __init__(self):
        self.q_table = np.zeros((5, 7, 4)) # q밸류를 저장하는 변수. 모두 0으로 초기화
        self.eps = 0.9
        self.alpha = 0.01
    
    def select_action(self, s):
        # eps-greedy로 액션을 선택
        x, y = s
        coin = random.random()
        if coin < self.eps:
            action = random.randint(0, 3)
        else:
            action_val = self.q_table[x, y,:]
            action = np.argmax(action_val)
        return action
    
    def update_table(self, transition):
        s, a, r, s_prime = transition
        x, y = s
        next_x, next_y = s_prime
        a_prime = self.select_action(s_prime) # s'에서 선택할 액션 (실제로 취할 액션이 아님)
        # SARSA 업데이트 식을 이용
        self.q_table[x, y, a] = self.q_table[x, y, a] + 0.1 * (r - self.q_table[next_x, next_y, a_prime] - self.q_table[x, y, a])
            
    def anneal_eps(self):
        self.eps -= 0.03
        self.eps = max(self.eps, 0.1)
    
    def show_table(self):
        # 학습이 각 위치에서 어느 액션의 q 값이 가장 높았는지 보여줌
        q_list = self.q_table.tolist()
        data = np.zeros((5, 7))
        for row_idx in range(len(q_list)):
            row = q_list[row_idx]
            for col_idx in range(len(row)):
                col = row[col_idx]
                action = np.argmax(col)
                data[row_idx, col_idx] = action
        print(data)

In [10]:
def main():
    env = GridWorld()
    agent = QAgent()
    
    for n_epi in range(1000): # 총 1000 에피소드 동안 학습
        done = False
        
        s = env.reset()
        while not done: # 한 Q에피소드가 끝날 때까지
            a = agent.select_action(s)
            s_prime, r, done = env.step(a)
            agent.update_table((s, a, r, s_prime))
            s = s_prime
        agent.anneal_eps()
        
    agent.show_table() # 학습이 끝난 결과를 출력

In [12]:
main()

[[1. 3. 0. 2. 0. 1. 3.]
 [3. 1. 0. 0. 2. 1. 2.]
 [0. 3. 0. 0. 0. 0. 3.]
 [2. 3. 2. 2. 0. 3. 2.]
 [2. 2. 3. 0. 0. 0. 0.]]
